# [CPython](python.org)

* Let's see how bindings work before going into C++ binding tools
* This is how CPython itself is implemented

> C reminder: `static` means visible in this file only

Let's keep things clean by putting this in a subdirectory (mostly so we can make more if we want to and not worry about reusing the setup.py name).

In [ ]:
mkdir pysimple

First, here's the code we want to wrap:

In [ ]:
%%writefile pysimple/pysimple.c

#include <Python.h>

float square(float x) {
    return x*x;
}

Okay, now we need to manually build the Python -> C -> Python conversion. *All* Python objects, regardless of type, are `PyObject*`. All the necessary conversions are provided using `Py*` functions.

* `PyArg_ParseTuple(PyObject*, format, *result)` will parse a tuple into a C.
* `PyFloat_FromDouble(double)` will convert a C item into a `PyObject*`.

In [ ]:
%%writefile -a pysimple/pysimple.c

static PyObject* square_wrapper(PyObject* self, PyObject* args) {
  float input, result;

  if (!PyArg_ParseTuple(args, "f", &input)) {
      return NULL;
  }
    
  result = square(input);
  return PyFloat_FromDouble(result);
}

Next, we need a `PyMethodDef`. This is a structure that holds functions. The structure is "name", wrapper function, argument type, and doc string. Since C doesn't know when an array ends, you use a null terminated row at the end to signal that that you are done. The argument type is an item from [this list](https://docs.python.org/3/c-api/structures.html); I'm excited about the new `METH_FASTCALL` in 3.7+...

Looking back, this could have been a *little* simpler with `METH_O`, which takes exactly one argument and therefore you'd avoid the Tuple.

In [ ]:
%%writefile -a pysimple/pysimple.c

static PyMethodDef pysimple_methods[] = {
   { "square", square_wrapper, METH_VARARGS, "Square function" },
   { NULL,     NULL,           0,             NULL }
};

Now we need a module structure. I won't go into it in great detail; `PyModuleDef_HEAD_INIT` is always there. Then the name of the module, then the module docstring (or NULL), then a size for subinterpreters (-1 to disable), the the methods you defined above.

In [ ]:
%%writefile -a pysimple/pysimple.c

static struct PyModuleDef pysimple_module = {
    PyModuleDef_HEAD_INIT, "pysimple", NULL, -1, pysimple_methods
};

Finally we have the last bit; you need to define a symbol with a specific name, `PyInit_<name>`, where `<name>` is the name of the module.

In [ ]:
%%writefile -a pysimple/pysimple.c

PyMODINIT_FUNC PyInit_pysimple(void) {
    return PyModule_Create(&pysimple_module);
}

To compile our module, we need to write a little setup.py. This is the correct way to compile it, and cross platform. You *can* just compile it by hand, which I've done here in the past, but it's very hard to get it right, and very system specific.

In [ ]:
%%writefile pysimple/setup.py

from setuptools import setup, Extension

module1 = Extension('pysimple',
                    sources=['pysimple.c']
                   )

setup(name='pysimple', ext_modules=[module1])

In [ ]:
!python pysimple/setup.py build_ext --inplace

### Run:

In [ ]:
import pysimple
pysimple.square(2.0)

In [ ]:
pysimple.__file__

## [Python limited API: PEP 384](https://www.python.org/dev/peps/pep-0384)

One possible benefit of building your own Python extensions this way is that you can use the *Python limited API*. This defines a reduced supset of functionality that you must stick to, but in return, you now can use a single compiled extension with *multiple* versions of Python, from a minimum (3.2 or later) to the current version (and future versions!).

Notice that sadly, [PyBuffer](https://jakevdp.github.io/blog/2014/05/05/introduction-to-the-python-buffer-protocol/) is not part of the limited API.

To use:
    
* Define `#define Py_LIMITED_API 0x03060000` at the top of your file (above the Python.h include). This number is the minimum python verison you want to support (`0x03030000`, or 3.2, is the earliest). Currenlty supported Python is 3.6+, just to let you know.
* Set `py_limited_api=True` in the setuptools Extension
* Make sure you don't use any of the (now missing) `Py*` functions that are not part of the limited API.

# Exercise

Try to convert the above code to use the Python limited API by making the two recommended changes. How did the file name change? What did you have to do to get it to load the new file? (note: you also need to delete the more-specific Python 3.8 file)